In [ ]:
from IPython.display import clear_output
from app import App

import warnings
warnings.filterwarnings('ignore')

%load_ext autoreload
%autoreload 2

SYMBOLS = ["BTCUSDT", "ETHUSDT", "SOLUSDT"]

app = App(symbols=SYMBOLS)
app.run()

## Socket

In [ ]:
import zmq
import pandas as pd
import numpy as np
import sqlalchemy
from datetime import datetime
import time

import asyncio


def connectionectionect_server(symbol):
    context = zmq.Context()
    socket = context.socket(zmq.SUB)
    socket.connectionect('tcp://127.0.0.1:8080')
    socket.setsockopt_string(zmq.SUBSCRIBE, symbol)
    return socket

def get_data(socket):
    data = socket.recv_string()
    _, date , opEn, high , low, close, volume, stop = data.split()
    date = datetime.fromtimestamp(float(date))
    d = {'open':float(opEn), 'low':float(low),
        'high':float(high), 'close':float(close), 'volume':float(volume)}
    return d, stop


while True:

    socket_1 = connect_server(symbol = "ETHUSDT")
    d1, stop = get_data(socket_1)
    print(d1)
    
    socket_2 = connect_server(symbol = "BTCUSDT")
    d2, stop = get_data(socket_2)
    print(d2)

# Session management

In [ ]:
import time 
import datetime

# durer session : 7 jours
# day trading

class SessionMangement:
    
    def __init__(self):
        self.durer_jrs = 7
        self.today = 0
        
        
    def new_session(self, start_date):
        self.start_date = start_date
        self.end_date = start_date
        
        
    def next_days(self):
        self.today =+ 1
        
        
    def start_session(self):
        ""
    
    def end_sessions(self):
        ""
        

In [ ]:
import pandas as pd
import numpy as np

arrays = [
    ["bar", "bar", "baz", "baz", "foo", "foo", "qux", "qux"],
    ["one", "two", "one", "two", "one", "two", "one", "two"],
]

tuples = list(zip(*arrays))

#index = pd.MultiIndex.from_tuples(tuples, names=["first", "second"])
index = pd.MultiIndex.from_tuples(tuples)

df = pd.DataFrame(np.random.randn(3, 8), index=["A", "B", "C"], columns=index)
df

In [ ]:
import pandas as pd
import numpy as np
import json
import websocket
from datetime import datetime
import time
from urllib.parse import urljoin, urlencode
import requests


def server_time():
    base_url = "https://testnet.binancefuture.com"
    path = "/fapi/v1/time"
    url = urljoin(base_url, path)
    servertime = requests.get(url).json()
    ServerTime = datetime.fromtimestamp(servertime['serverTime']/1000)
    return ServerTime


class App:
    
    def __init__(self):
        self.url = "wss://stream.binancefuture.com/ws/"
        self.data = pd.DataFrame()
        
        
    def on_open(self, ws):
        print("open connection")
        INT = '5m'
        params = json.dumps({
            "method" : "SUBSCRIBE",
            "params" : ['btcusdt@kline_'+INT, 'ethusdt@kline_'+INT, 'solusdt@kline_'+INT]
        })
        ws.send(params)
        
        
    def on_message(self, ws, data):
        data = json.loads(data)
        print(data)
        add = {
            'open' : data['k']['o'],
            'high' : data['k']['h'],
            'low' : data['k']['l'],
            'close' : data['k']['c'],
            'volume' : data['k']['v'],
            'kclosed' : data['k']['x'],
            'symbol' : data['s']
        }
        time = data['k']['T']
        time = datetime.fromtimestamp(time/1000)
        add['closeTime'] = time
        
        startime = datetime.fromtimestamp(data['k']['t']/1000)
        add['startime'] = startime
        
        currentTime = server_time()
        
        date = time.strftime("%Y-%m-%d %H:%M")
            
        kclose = data['k']['x']
        self.data = self.data.append(
            pd.DataFrame(add, index = [currentTime])
                )
        #if kclose:
        #self.data = self.data.resample('1min').last()
        display(self.data)
        
    
    def on_error(self, ws, error):
            print("Error")
            print(error)
            #raise error
    
    def run(self):
        ws = websocket.WebSocketApp(self.url,
                            on_open = self.on_open,
                            on_message = self.on_message,
                            on_error = self.on_error
                            )
        ws.run_forever()




app = App()
app.run()

In [ ]:
app.data.head()

In [ ]:
app.data.resample('1min').last()

# Binance Websocket

In [ ]:
import asyncio
import sys
import sqlite3
import aiosqlite
import json

connection = sqlite3.connect("./data.db")
cursor = connection.cursor()

cursor.execute("DROP TABLE IF EXISTS trades")
cursor.execute(""" CREATE TABLE trades (
                            id int PRIMARY KEY,
                            time int,
                            quantity int,
                            price float) """)
cursor.execute("CREATE INDEX index_time ON trades(time)")

connection.commit()
connection.close()